In [39]:
import pandas as pd
import sqlite3
from sklearn.model_selection import train_test_split
import nltk
import string
from sklearn.ensemble import RandomForestClassifier
from sklearn.multiclass import OneVsRestClassifier
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import SnowballStemmer
nltk.download('punkt')
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import precision_score, recall_score, precision_recall_curve, accuracy_score,hamming_loss
import numpy as np
from sklearn.preprocessing import Normalizer
from sklearn.base import BaseEstimator, TransformerMixin, ClassifierMixin
import joblib

[nltk_data] Downloading package punkt to C:\Users\zxc
[nltk_data]     ghoul\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [4]:
db_name = 'recipe.db'
conn = sqlite3.connect(db_name)

table_name = 'recipe'

#забираем доступные категории
categories_sql = f"select c.id, c.name_cat from category c " \
                 f"join recipe_categories rc on " \
                 f"c.id = rc.cat_id " \
                 f"GROUP by c.id " \
                 f"HAVING count(*) > 100 " \
                 f"ORDER BY c.id"

loaded_categories = pd.read_sql(categories_sql, conn)

ids_cat = []
for i in range(len(loaded_categories)):
    ids_cat.append(str(loaded_categories.id[i]))

#забираем способ приготовления рецепта с категориями из доступных с id категорий
sql = f"SELECT r.manual, (SELECT group_concat(rc.cat_id , ', ') " \
      f"from recipe_categories rc " \
      f"WHERE rc.recipe_id = r.id AND " \
      f"rc.cat_id in ({', '.join(ids_cat)})) " \
      f"AS categories from recipe r"

loaded_data = pd.read_sql(sql, conn)

#pandas data frame
conn.close()

In [5]:
ids_cat = []
for i in range(len(loaded_categories)):
    ids_cat.append(str(loaded_categories.id[i]))

print(ids_cat)

['1', '2', '4', '5', '7', '8', '14', '16', '17', '20', '25', '27', '30', '33', '37', '43', '51', '62', '67', '68', '71', '90', '99', '103', '105', '139', '140', '150', '158', '257']


In [6]:
for i in range(len(ids_cat)):
    val = []
    for j in range(len(loaded_data)):
        spl = str(loaded_data.categories[j]).split(', ')
        val.append(int(str(ids_cat[i]) in spl))
    loaded_data.insert(loc=len(loaded_data.columns) , column=ids_cat[i], value=val)

In [7]:
#делим dataset на тренировочный и тестовый
x_train,x_test,y_train,y_test = train_test_split(loaded_data["manual"], loaded_data[ids_cat],test_size=0.3,random_state=42)

In [15]:
snowball = SnowballStemmer(language="russian")
russian_stop_words = stopwords.words("russian")

def tokenize_sentence(sentence: str, remove_stop_words: bool = True):
    tokens = word_tokenize(sentence, language="russian")
    tokens = [i for i in tokens if i not in string.punctuation]
    if remove_stop_words:
        tokens = [i for i in tokens if i not in russian_stop_words]
    tokens = [snowball.stem(i) for i in tokens]
    return tokens

In [29]:
class CustomThreshold(BaseEstimator, ClassifierMixin):
    """ Custom threshold wrapper for binary classification"""
    def __init__(self, base, threshold=0.5):
        self.base = base
        self.threshold = threshold
    def fit(self, *args, **kwargs):
        self.base.fit(*args, **kwargs)
        return self
    def predict(self, X):
        probabilities = self.base.predict_proba(X)
        predictions = np.zeros_like(probabilities)
        for i in range(len(loaded_categories)):
            predictions[:, i] = (probabilities[:, i] > self.threshold).astype(int)
        return predictions

In [34]:
model_pipeline = Pipeline([
    ("vectorizer",  TfidfVectorizer(tokenizer=lambda x: tokenize_sentence(x, remove_stop_words=True), max_features=768)),
    ("normalizer", Normalizer()),
    ("model",  OneVsRestClassifier(RandomForestClassifier()))
]
)

In [36]:
clf = CustomThreshold(model_pipeline, 0.45)
clf.fit(x_train, y_train)

C:\Users\zxc ghoul\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\feature_extraction\text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


CustomThreshold(base=Pipeline(steps=[('vectorizer',
                                      TfidfVectorizer(max_features=768,
                                                      tokenizer=<function <lambda> at 0x000001CE8B2F2EE0>)),
                                     ('normalizer', Normalizer()),
                                     ('model',
                                      OneVsRestClassifier(estimator=RandomForestClassifier()))]),
                threshold=0.45)

In [37]:
accuracy_score(y_test,clf.predict(x_test))

0.12005717008099095

In [38]:
precision_score(average='weighted', y_true=y_test, y_pred=clf.predict(x_test))

C:\Users\zxc ghoul\AppData\Local\Programs\Python\Python39\lib\site-packages\sklearn\metrics\_classification.py:1344: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


0.6798496725963014

In [42]:
joblib.dump(clf, "./clf.joblib")

PicklingError: Can't pickle <function <lambda> at 0x000001CE8B2F2EE0>: it's not found as __main__.<lambda>